# Example for utilization of powerfactory-tools -- Control

In [1]:
%load_ext autoreload
%autoreload 2

## Select an compatible PowerFactory Interface
- Import the PowerFactoryInterface (in this example: that is compatible with PowerFactory in version 2024)
- Specify additional PowerFactory configuration
- Name the PowerFactory project

In [2]:
import logging
import math
import pathlib
import time
import typing as t
from collections.abc import Sequence

from powerfactory_tools.versions.pf2024 import PowerFactoryInterface
from powerfactory_tools.versions.pf2024.interface import ValidPFValue
from powerfactory_tools.versions.pf2024.interface import ValidPythonVersion
from powerfactory_tools.versions.pf2024.types import CalculationCommand
from powerfactory_tools.versions.pf2024.types import PFClassId
from powerfactory_tools.versions.pf2024.types import PowerFactoryTypes as PFTypes

# PowerFactory configuration
PF_SERVICE_PACK = 2  # mandatory
PF_USER_PROFILE = ""  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PF_PYTHON_VERSION = ValidPythonVersion.VERSION_3_12  # python version of local code environment must match the python version of PowerFactory API


# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\project_name"
EXPORT_PATH = pathlib.Path("control_action_results")

<>:23: SyntaxWarning: invalid escape sequence '\P'
<>:23: SyntaxWarning: invalid escape sequence '\P'
D:\Users\s9842011\AppData\Local\Temp\ipykernel_17584\1069366573.py:23: SyntaxWarning: invalid escape sequence '\P'
  PROJECT_NAME = "7 Tests\PowerFactory-Tools-Dev"  # may be also full path "dir_name\project_name"


## Define control routine to create new grid variant and add loads to terminals

In [3]:
def create_load(pfi: PowerFactoryInterface, *, grid: PFTypes.Grid, load_data: dict[str, t.Any]) -> PFTypes.Load | None:
        terminal = load_data.pop("terminal")
        load_name = load_data.pop("name")
        logging.info(f"Create new load '{load_name}' at terminal '{terminal.loc_name}' ...")  # noqa: G004

        term_cubic = terminal.GetContents("*." + PFClassId.CUBICLE.value)[0]
        # it is necessary to add the copy exactly on a bus, otherwise the field won't have the attribute 'cterm'
        new_cubicle = terminal.AddCopy(term_cubic, "cub_new_")
        data = {"bus1": new_cubicle}
        load_data.update(data)

        load = pfi.create_object(
            name=load_name,
            class_name=PFClassId.LOAD.value,
            location=grid,
            data=load_data,
        )
        return t.cast("PFTypes.Load", load) if load is not None else None

def routine_create_new_study_case_and_add_loads(
        pfi: PowerFactoryInterface,
        *,
        study_case_name: str,
        grid_variant_name: str,
        grid: PFTypes.Grid,
        loads_data: Sequence[dict[str, ValidPFValue]],
    ) -> Sequence[PFTypes.Load] | None:
        """Use this routine to create a new study case (based on the previously active one) and add new loads to given terminals.

        Args:
            study_case_name (str): new study case name
            grid_variant_name (str): new grid variant name
            grid (PFTypes.Grid): grid to which the loads belongs
            loads_data (Sequence[dict[str, ValidPFValue]]): list of dictionaries with load data

        Returns:
            Sequence[PFTypes.Load]: list of newly added loads
        """
        ## Create new study case and grid variant
        # Get active study case and grid variants
        active_sc = pfi.app.GetActiveStudyCase()
        default_active_variants = pfi.grid_variants(only_active=True)

        # Create new study case
        new_sc = pfi.study_case_dir.AddCopy(active_sc)
        new_sc.loc_name = study_case_name
        pfi.switch_study_case(study_case_name)

        # Create and activate new grid variant
        activation_time = int(math.floor(time.time()))  # now in seconds
        new_variant = pfi.create_grid_variant(name=grid_variant_name, data={"tAcTime": activation_time}, force=True)
        pfi.switch_grid_variant(grid_variant_name)

        ## Add new loads
        loads = [create_load(pfi, grid=grid, load_data=load_data) for load_data in loads_data]

        ## Reactivate old (already existing) and new grid variants
        pfi.deactivate_grid_variants()
        # reactivate old active variants
        for variant in default_active_variants:
            pfi.activate_grid_variant(variant)
        # reactivate new variant at the end
        pfi.activate_grid_variant(new_variant)

        return loads





## Define routine to run Automatic Graph Layout Command

In [ ]:
def create_sgl_layout_selection(
        pfi: PowerFactoryInterface,
        *,
        data: Sequence[PFTypes.Element],
        location: PFTypes.StudyCase | None = None,
    ) -> PFTypes.Selection | None:
        if location is None:
            sc = pfi.app.GetActiveStudyCase()
        selection = pfi.create_object(
            name="SGL-Layout-Selection",
            class_name=PFClassId.SELECTION.value,
            location=sc,
        )

        if selection is None:
            return None
        selection = t.cast("PFTypes.Selection", selection)

        selection.AddRef(data)

        return selection

def run_layout(pfi: PowerFactoryInterface, *, selection: PFTypes.Selection | None = None) -> bool:
    """This function gets command GRAPHIC_LAYOUT_TOOL, adjust and executes it."""
    logging.debug("Start automatic layout applicaton ...")

    layout: PFTypes.CommandSglLayout = pfi.app.GetFromStudyCase(CalculationCommand.GRAPHIC_LAYOUT_TOOL.value)
    layout.iAction = 1
    layout.orthoType = 1
    layout.nodeDispersion = 0
    layout.insertionMode = 0
    layout.neighborhoodSize = 3
    layout.neighborStartElems = selection
    # layout.cFreezeMode = 0
    # TODO unfreeze desktopPage
    try:
        layout.Execute()
    except RuntimeError:
        logging.warning("Automatic layouting failed.")
        return False
    return True

## Execute control action using a controller instance

In [4]:
_project_name = PROJECT_NAME.split("\\")
full_export_path = pathlib.Path().cwd() / EXPORT_PATH / _project_name[-1]

# Configure logging to output to the notebook's standard output
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

with PowerFactoryInterface(
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PF_PYTHON_VERSION,
    project_name=PROJECT_NAME,
    logging_level=logging.INFO,
    # log_file_path=full_export_path / pathlib.Path("pf_control.log"),  # noqa: ERA001
) as pfi:
    logging.info("3_Bus : Run control example 'Add loads' ... ")

    study_case_name = "3_Bus"
    grid_name = "HV_3_Bus"

    # From active study case: get all relevant! terminals
    logging.info("3_Bus : From active study case: get all terminals ...")
    study_case = pfi.switch_study_case(study_case_name)
    terminals = pfi.terminals(calc_relevant=True)

    # Specify load data (connected terminal, load name, further load data)
    loads_data = [{"name": "NewLoad1", "terminal": terminals[1], "plini": 1, "qlini": 0.5}, 
                  {"name": "NewLoad2", "terminal": terminals[2], "plini": 2, "qlini": 1}]

    loads = routine_create_new_study_case_and_add_loads(pfi, study_case_name="NewLoadsAdded", grid_variant_name="NewLoadsAdded", grid=pfi.grid(grid_name), loads_data=loads_data)

    # selection = create_sgl_layout_selection(pfi, data=loads)
    # success = run_layout(pfi, selection=selection)

    logging.info("3_Bus : Run control example 'Add loads' ... Done")

2025-01-24 15:49:43 interface.py:114 Starting PowerFactory Interface...
2025-01-24 15:50:11 interface.py:122 Starting PowerFactory Interface... Done.


2025-01-24 15:50:11,954 - INFO - 3_Bus : Run control example 'Add loads' ... 
2025-01-24 15:50:11,955 - INFO - 3_Bus : From active study case: get all terminals ...


2025-01-24 15:50:11 interface.py:526 Study_case 3_Bus is already inactive.


2025-01-24 15:50:12,076 - INFO - Create new load 'NewLoad1' at terminal 'Node_2' ...
2025-01-24 15:50:12,083 - INFO - Create new load 'NewLoad2' at terminal 'Node_3' ...
2025-01-24 15:50:12,090 - INFO - 3_Bus : Run control example 'Add loads' ... Done


2025-01-24 15:50:12 interface.py:291 Closing PowerFactory Interface...
2025-01-24 15:50:13 interface.py:298 Closing PowerFactory Interface... Done.
